<a href="https://colab.research.google.com/github/juancarloscaraguay1-lang/-rbol-de-decisi-n-CART/blob/main/Copia_de_Cart_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 31005_A2: Programming the decision tree (CART) algorithm

Hong Kung(12431868) & Weixiang Gao(12765653)

Video link: https://www.youtube.com/watch?v=koAg_y_l3VY&feature=youtu.be
<br>

## Introduction



El análisis de datos mediante la construcción de modelos se ha utilizado ampliamente en diversos campos, especialmente para analizar los diferentes aspectos de los productos en el mercado. El diamante es uno de los commodities de inversión más populares en los últimos años, el rango de precios es muy amplio según las diferencias de calidad en muchos aspectos.<br>

Por lo tanto, el objetivo de nuestro proyecto es simple: programar un árbol de decisión para ***clasificar*** el rango de precios de los diamantes (bajo, medio, alto). Para alcanzar este objetivo, utilizamos el **conjunto de datos (diamonds.csv)** de Kaggle para entrenar y probar nuestro modelo de árbol de decisión. La elección más adecuada en este caso es un árbol binario de estructura simple. Optamos por usar el Árbol de Clasificación y Regresión (CART). Dado que todos los datos que usamos son discretos, se creará un ***árbol de clasificación*** en lugar de un árbol de regresión. El detalle del algoritmo del árbol y la construcción del modelo se discutirán en la parte siguiente.<br>

La entrada de nuestro proyecto es el conjunto de datos de diamantes que se utiliza para el entrenamiento y la prueba. La salida de nuestro proyecto es el árbol de decisión CART y su precisión en la prueba. Y finalmente hablaremos sobre el problema ético relacionado con nuestro proyecto.

## Exploración

### Identificar el desafío

El principal desafío de este proyecto es que CART solo admite divisiones binarias, lo que significa que los datos solo se pueden dividir en dos partes (respuesta sí al divisor, respuesta no al divisor); el desafío principal es diseñar el modelo de estos problemas y determinar los divisores bajo la teoría básica de CART.


### Preparación

Para iniciar nuestro programa, necesitamos importar todas las bibliotecas que necesitamos para este proyecto.

In [ ]:
# Procesamiento ndarray
import numpy as np
# Procesamiento Dataframe
import pandas as pd

# para seleccionar aleatoriamente datos de entrenamiento/prueba de un dataframe
import random
# "pretty-print" una estructura de datos arbitraria (un objeto dict de un árbol en nuestro caso) de forma más clara
from pprint import pprint

entonces, carguemos y verifiquemos el dataframe de origen.

In [ ]:
url = 'https://raw.githubusercontent.com/STKKKKK/UTS_ML_2019_ID12431868/master/31005_ML_A2/diamonds.csv'

df = pd.read_csv(url)
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


El dataframe contiene los siguientes atributos :

precio : precio en dólares estadounidens($326--$18,823)

carat : peso en quilates (0.2--5.01)

corte : calidad del corte (Regular, Bueno, Muy Bueno, Premium, Ideal)

color : color del diamante, de J (peor) a D (mejor)

claridad : medida de cuán claro es el diamante: I1 (peor), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (mejor)

x : longitud en mm (0--10.74)

y : ancho en mm (0--58.9)

z : profundidad en mm (0--31.8)

profundidad : porcentaje total de profundidad = z / media(x, y) = 2 * z / (x + y) (43--79)

tabla : ancho de la parte superior del diamante relativo al punto más ancho (43--95)

In [ ]:
#  una visión general

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


In [ ]:
Y afortunadamente, no hay valores nulos en el conjunto de datos.

A partir de ahora, necesitamos decidir qué atributos son más 'significativos' en este proyecto. Según el estándar oficial de clasificación de diamantes (GIA), los 4 factores más importantes (4C) son: peso en quilates, corte, color y claridad.
Por lo tanto, mantendremos estos atributos y eliminaremos otras columnas del dataframe.

In [ ]:
df = df.drop(['Unnamed: 0','depth','table','x','y','z'],axis=1)

Además, podemos convertir fácilmente los datos categóricos en algunos valores numéricos ya que estos datos son 'escalas' de un diamante. Según el GIA de diamantes, usamos toda la escala de clasificación.

In [ ]:
grade = {'cut': ['Fair','Good','Very Good','Premium','Ideal'],
         'color': ['J','I','H','G','F','E','D'],
         'clarity': ['I3','I2','I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF','FL']}
         # ordered from worst to best

for key, scale in grade.items():
   for index in range(len(scale)):
      df[key] = df[key].map(lambda x: index if x == grade[key][index] else x)

Por supuesto, como mencionamos, también necesitamos convertir el precio en dólares estadounidenses en las categorías baja, media y alta. Antes de eso, necesitamos hacer un vistazo general a la distribución del valor de los precios.

In [ ]:
df.describe()

,carat,cut,color,clarity,price
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,2.904097,3.405803,5.051020,3932.799722
std,0.474011,1.116600,1.701105,1.647136,3989.439738
min,0.200000,0.000000,0.000000,2.000000,326.000000
25%,0.400000,2.000000,2.000000,4.000000,950.000000
50%,0.700000,3.000000,3.000000,5.000000,2401.000000
75%,1.040000,4.000000,5.000000,6.000000,5324.250000
max,5.010000,4.000000,6.000000,9.000000,18823.000000


En este caso, tomamos los valores de 1200 y 4000 como el valor límite de la clase de precio.

In [ ]:
df['price'] = df['price'].map(lambda x: 'low' if x<1200 else
                              ('high' if x>4000 else 'medium'))

Ahora, finalmente vamos a revisar los datos que se utilizaron únicamente para este programa.

In [ ]:
df.head()

,carat,cut,color,clarity,price
0,0.23,4,5,3,low
1,0.21,3,5,4,low
2,0.23,1,5,6,low
3,0.29,3,1,5,low
4,0.31,1,0,3,low


Además, tenemos que decidir aleatoriamente los índices de los dataframes de Entrenamiento y Prueba; obtendremos 200 muestras para cada uno.

In [ ]:
random.seed(1)

indices = df.index.tolist()
test_indices = random.sample(population=indices, k=200)

test_df = df.loc[test_indices]
train_df = df.drop(test_indices)

Finalmente, convertiremos el dataframe de entrenamiento a un array 2D de numpy para un procesamiento más conveniente y lo asignaremos a la variable 'data'

In [ ]:
data = train_df.values

Tenga en cuenta que deberíamos tener estas variables que usaríamos mucho en nuestro programa :

df
test_df
train_df
data

### Modelado

 ![flowchart](https://raw.githubusercontent.com/STKKKKK/UTS_ML_2019_ID12431868/master/31005_ML_A2/A2_flowChart.png)

Según el diagrama de flujo, se muestra que se necesita lograr un bucle de división de datos. Por lo tanto, elegiremos un árbol de decisión (CART) para manejar esto. Por supuesto, el árbol de decisión ID3 y el árbol de decisión C4.5 también pueden ser enfoques alternativos. <br>  
El concepto clave del árbol de decisión CART es identificar la división basándose en el Gini-Score, que es un valor que describe la impuridad de un conjunto de datos. Cuanto más bajo es el Gini-Score de un conjunto de datos, más probable es que los datos pertenezcan a la misma clase.<br>  
En este caso, necesitamos encontrar la mejor división entre todas las divisiones posibles calculando la ganancia de Gini más baja de estas divisiones.<br>  
También hemos encontrado que el término ***'split' o 'splitter'*** se menciona mucho. ¡Es una buena manera de definirlo como un objeto abstracto!

## Metodología

### Un objeto dividido

In [ ]:
class Split():

    def __init__(self, data, column, value):
        """
        column: SPLIT attribute (column index of data)
        value: SPLIT value
        data_below/above: data set which sperated by this SPLIT
        """
        self.column = column
        self.value = value
        self.data_below = data[data[:, self.column] < value]
        self.data_above = data[data[:, self.column] >= value]


    def formatt(self):
        """
        Represent the SPLIT object in a more intuitive way
        """
        return "{} < {}".format(df.columns[self.column], self.value)


    def gini_gain(self):
        """
        Calculate the Gini-gain of the SPLIT
          consider this value uses both sides data sets' Gini-Score
        """
        def gini(data):
            """
            Calculate the Gini-Score of a data set
            """
                # obtener el recuento de grados de precio únicos
            _, counts = np.unique(data[:, -1], return_counts=True)
            p = counts / counts.sum()

                # Fórmula del índice de Gini:
            Gini = 1 - sum(p ** 2)

            return Gini

        n_data = len(self.data_below) + len(self.data_above)
        p_below = len(self.data_below) / n_data
        p_above = len(self.data_above) / n_data

            # Fórmula de la ganancia de Gini:
        Gini_gain = p_below*gini(self.data_below)+ p_above*gini(self.data_above)

        return Gini_gain

In [ ]:
# pequeña prueba

Split(data, 3, 1.8).gini_gain()

np.float64(0.6655809650983266)

### Obtener el mejor corte

In [ ]:
def get_best_split(data):
    """
    1. Initially, we need to get all the potential splits of the data set.
    This can be done by finding ALL the Different data points in the
    data set, and the Split will take the avarage value of each two points
    in order to let both points settled away from the Split as possible.

    2. After that, we will find the best Split based on its gini_gain.
    """
    potential_splits = []
    _, n_columns = data.shape

    for column in range(n_columns -1):   # no necesitamos la columna de precios
        unique_values = np.unique(data[:, column])

        for index in range(len(unique_values)):
                # El primer valor no será procesado
                # porque no tiene un valor anterior
            if index > 0:
                current = unique_values[index]
                previous = unique_values[index -1]
                split = Split(data, column, (current + previous)/2)
                potential_splits.append(split)


      # iterar sobre todas las posibles divisiones
       # un valor de gini_gain nunca será mayor que 1, así que podemos comenzar desde aquí
    max_gini = 1

    for split in potential_splits:
          if split.gini_gain() <= max_gini:
            max_gini = split.gini_gain()
            best_split = split

    return best_split

In [ ]:
# pequeña prueba: esta es la primera división de datos

sp = get_best_split(data)
sp.formatt()

'carat < 0.495'

### El árbol de decisión

In [ ]:
def train(data, max_depth, depth = 0):
    """
    This function aimed to build the CART decision tree by training the data.
    This is a Recursive Function!
    max_depth: can be set to define the maximum depth of tree

    The tree will be a nested dict object, consider it is built up by sub trees
     with three nodes: {root: [true_leaf, false_leaf]}
    """
    price_column = data[:, -1]
    unique_classes, count = np.unique(price_column, return_counts=True)

        # si el grado de precio es único, o alcanzan la profundidad máxima establecida
    if(len(unique_classes) == 1) or (depth == max_depth):
           # clasificar
        end_node = unique_classes[count.argmax()]

        return end_node

    else:
        depth += 1    # contar hasta alcanzar la profundidad máxima

        best_split = get_best_split(data)

           # construir el árbol
        root = best_split.formatt()
        sub_tree = {root: []}
        true_leaf = train(best_split.data_below, max_depth, depth)
        false_leaf = train(best_split.data_above, max_depth, depth)

           # asegúrate de que la misma hoja no se represente al mismo tiempo
        if true_leaf != false_leaf:
            sub_tree[root].append(true_leaf)
            sub_tree[root].append(false_leaf)
        else: sub_tree = true_leaf

        return sub_tree

In [ ]:
def test(data, tree):
    """
    The function aimed to test the data by using a CART decision tree.
    This is a Recursive Function!
    """
    root = list(tree.keys())[0]    # obtener el nodo raíz del subárbol
    column, less_than, value = root.split()

    if data[column] < float(value):   # 'value' necesita convertirse de str
        leaf = tree[root][0]
    else: leaf = tree[root][1]

    if type(leaf) == dict:    # todavía no he llegado al nodo final de todo el árbol
        return test(data, leaf)
    else: return leaf     # llegar finalmente al nodo final

## Evaluación

### Rendimiento

**Primero, hagamos una prueba sencilla para comprobar si la función de prueba de árbol funciona:**

In [ ]:
print(test_df.iloc[14],'\n')
print("The price is '{}'".format(test(test_df.iloc[14], tree)))

carat      1.03
cut           4
color         5
clarity       4
price      high
Name: 13759, dtype: object 



NameError: name 'tree' is not defined

La prueba parece funcionar y podemos descubrir que la decimocuarta prueba (de un total de 200 muestras) es correcta.
<br>
<br>


**Ahora, podemos entrenar los datos y crear un árbol de decisiones con una profundidad de 5.**

In [ ]:
tree = train(data, 5)

pprint(tree)  # use 'pretty-print'

**Después de eso, aplicaremos las muestras de prueba para realizar la prueba. Y finalmente, podremos calcular la precisión.**

In [ ]:
def accuracy(data, tree):

     # crear una nueva columna de resultados de pruebas
  test_df['test_result'] = test_df.apply(test, axis=1, args=(tree,))

     # calcular la media de los valores booleanos de corrección
     #  (1 si es correcto, 0 si es incorrecto), obtenemos la exactitud del 'árbol'
  accuracy = (test_df.test_result == test_df.price).mean()

  print(test_df.head())
  print('\n','\n','The Accuracy of our CART decision tree is {}'.format(accuracy))


In [ ]:
accuracy(test_df, tree)

<br>
<br>

**Entrenemos y probemos nuevamente con un árbol de profundidad 6.**

In [ ]:
tree_depth_6 = train(data, 6)
accuracy(test_df, tree_depth_6)

### Eficiencia

In [ ]:
%timeit tree_depth_of_4 = train(data, 4)

In [ ]:
%timeit tree_depth_of_5 = train(data, 5)

In [ ]:
%timeit tree_depth_of_6 = train(data, 6)

La eficiencia general del tiempo no es tan satisfactoria como la precisión, por eso siempre necesitamos limitar la profundidad del árbol según diferentes situaciones. (Cuanto más grande/profundo sea el árbol, mayor será la precisión pero menor la eficiencia)

### Estudio comparativo

*El* concepto principal del algoritmo CART es la ganancia de Gini. De hecho, hay otros dos tipos de árboles de decisión ampliamente utilizados que usan conceptos diferentes y tienen sus propias limitaciones:
- El árbol de decisión ID3 es un algoritmo voraz, la toma de decisiones se basa en la ganancia de información, buscando la división con la menor entropía general (un valor que describe el caos de la información), lo que significa la mayor ganancia de información. Sin embargo, el árbol de decisión ID3 no puede manejar datos continuos.
- El árbol de decisión C4.5 es una versión avanzada de ID3, en lugar de buscar la ganancia de información, se enfoca en encontrar la razón de ganancia de información. El algoritmo C4.5 soporta tanto datos continuos como discretos, al igual que CART, pero todavía tiende a ajustarse mejor a datos discretos.
- CART es lo que estamos programando en este proyecto. La limitación más significativa de CART son las divisiones binarias, lo que puede llevar al problema de sobreajuste de los datos.

También podemos usar la biblioteca sklearn para implementar todos los algoritmos de árboles de decisión. Ten en cuenta que también podemos usar los datos de iris para hacer el entrenamiento y las pruebas.

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
import graphviz

iris = load_iris()
X = iris.data
y = iris.target
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X,y)

 # gráfico de datos dot
data = tree.export_graphviz(clf, out_file=None,
                                 feature_names=iris.feature_names,
                                 class_names=iris.target_names,
                                 filled=True,
                                 rounded=True,
                                 special_characters=True)

graph = graphviz.Source(data)
graph.view()

## Conclusión

Para concluir, dentro de las limitaciones de tiempo y conocimiento, aprendimos cómo superar las dificultades durante el desarrollo y nos sentimos muy satisfechos cuando el modelo se construyó con éxito y pudo alcanzar el objetivo del proyecto; encuentra el rango de precio de un diamante principalmente analizando el peso en quilates, el corte, el color y la claridad de un diamante. La precisión es suficientemente aceptable (aproximadamente 0,95 cuando la profundidad del árbol es 5). Por supuesto, también podemos aprender cómo construir un CART de regresión y existen varias mejoras en el algoritmo CART actual, como cómo manejar el sobreajuste de los datos, ya que esto requiere un entendimiento mucho más profundo de la estructura del modelo de árboles y del campo completo del aprendizaje automático.  

## Ética

Las consideraciones éticas son inevitables a medida que las computadoras juegan roles cada vez más importantes en la vida de los seres humanos. Existen diferentes posturas sobre la ética con respecto a las máquinas. Algunos podrían argumentar que la ética de las máquinas existe porque los humanos son máquinas y los humanos tienen ética. Otros podrían argumentar que la ética de las máquinas no existe porque la ética es simplemente una expresión emocional y las máquinas no pueden tener emociones (Moor, 2006). Aunque la discusión sobre esto puede llevarnos rápidamente a cuestiones filosóficas, no podemos ignorarla. No podemos—ni debemos—evitar la consideración de la ética de las máquinas en el mundo tecnológico actual, afirmó Moor (2006). Según el enfoque kantiano basado en el deber, no son las cosas las que afectan a las personas, sino las personas las que influyen en las cosas. Somos nosotros, los seres humanos, quienes estamos construyendo el mundo real. En el proceso de conocer las cosas, las personas son más importantes que las propias cosas. Las personas son moralmente autónomas. Aunque el comportamiento humano está limitado por la causalidad objetiva, las personas se convierten en seres humanos porque tienen libertad moral, pueden trascender la causa y el efecto, y pueden ser responsables de sus acciones. Por lo tanto, es importante determinar la ética correcta, y la ética debe tener autoridad suprema. La ética mide si la acción en sí misma está en consonancia con los estándares éticos, sin considerarla solo por el resultado de la felicidad.
Considerando que nuestro proyecto es el modelo para analizar los factores determinantes del precio de un diamante, sirve como guía para los clientes que no son profesionales en este campo. Podría ayudarlos a identificar si los diamantes que compran valen el precio correspondiente. El mercado está determinado por la relación entre la oferta y la demanda, por lo que nuestro modelo no destruirá el equilibrio. Según nuestro conocimiento, es poco probable que nuestro proyecto sea mal utilizado.

## Presentación de video


https://www.youtube.com/watch?v=koAg_y_l3VY&feature=youtu.be

## Referencias

LUMERA: The 4 C's Of Diamonds, & So Much More https://www.lumeradiamonds.com/diamond-education/index <br>
Moor, J. (2006). The Nature, Importance, and Difficulty of Machine Ethics. IEEE Intelligent Systems, 21(4), pp.18-21.<br>

**Conclusión**

La aplicación del árbol de decisión (CART) en Ecuacorriente, específicamente en el área de trabajo donde me desempeño que es la Unidad de Control de Calidad del Departamento de Gestión de depósitos de relaves GDR, la cual se dedica al control de calidad de la construcción de la presa de relaves, puede ser muy útil para el control de la compactación de cada capa de material, así como determinar la densidad seca del material compactado.

Tomando en cuenta el árbol de decisión CART es un modelo que clasifica o predice resultados basándose en diferentes variables, se puede procesar datos en base al tipo de compactador que se disponga, el número de ciclos de compactación, el contenido de humedad del material, el espesor de la capa y generar reglas especificas, lo cual nos podria ayudar a determinar cual es la densidad seca del material compactado y si cumple con las valores establecidos en las especificaciones técnicas.